In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

# Цена

Парсить цену можно двумя способами: доставать из раздела, который выводит цену на экран или из раздела 'itemprop', в котором тоже записана цена. В первом случае есть проблема с количеством символов: можно не вывести часть цифр, если значение больше миллиарда (таких квартир, согласно ЦИАНу немало). Во втором случае тоже могут возникнуть проблемы, так как в этом разделе значение не всегда отображается верно.
Остановимся на первом варианте, но с небольшой попроавкой. В большинстве случаев, если цена более миллиарда, то она записана и в рублях, и в долларах, а между двумя значениями ставится разделительный знак. Этот знак ставится определенного цвета, который закордирован значением '808080'. Значит, если в коде страницы будет этот знак, то будем брать все значение после него, т. е. все значение в рублях. Если знака нет, то будем брать последние значения, то есть парсить цену до миллиарда. 

In [4]:
def find_element(element, list_element):
    try:
        index_element = list_element.index(element)
        return index_element+1
    except ValueError:
        return 0

In [5]:
def getPrice(flat_page):
    try:
        price = flat_page.find('div', attrs={'class':'object_descr_price'})
        price = re.split('<div>|за м<sup>2</sup>|\n|80800|руб|\W', str(price))
        price = [i for i in price if i.isdigit()]
        indextilda = find_element('808080',price)
        price1 = "".join([i for i in price if i.isdigit()][(indextilda):])
        return int(price1)
    except:
        return ()

# Количество комнат

In [6]:
def getRoom(flat_page):
    try: 
        rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
        rooms = html_stripper(rooms)
        room_number = ''
        for i in re.split('-|\n', rooms):
            if 'комн' in i:
                break
            else:
                room_number += i
        room_number = "".join(room_number.split())
        return room_number
    except:
        return ()

# Общая площадь

#### Площади берем из раздела 'table'. Есть небольшая деталь, что площадь может быть записана целым числом или с дробной частью. Для этого мы сравним количество символов в строке и количество цифр в строке. Если символов больше, то значит есть запятая, отделяющая дробную часть. В этом случае полученное числовое значение надо дополнительно разделить на 10. 

In [7]:
def getTotsp (flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        tspace = re.split('Общая площадь|Площадь комнат', table)[1]
        tspace1 = re.split('-|\n', tspace)
        tspace1 = re.split('\xa0|м2',tspace1[2])
        tspace2 = "".join([i for i in tspace1[0] if i.isdigit()])
        if len(tspace1[0]) != len(tspace2):
            totsp = int(tspace2)/10
        else:
            totsp = int(tspace2)
        return totsp
    except: 
        return ()

# Общая жилая площадь

In [8]:
def getLivesp (flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        lspace = re.split('Жилая площадь|Площадь кухни', table)[1]
        lspace1 = re.split('-|\n', lspace)
        lspace1 = re.split('\xa0|м2',lspace1[2])
        lspace2 = "".join([i for i in lspace1[0] if i.isdigit()]) #В случае нецелых значений 
        if len(lspace1[0]) != len(lspace2):#В случае нецелых значений площади
            livesp = int(lspace2)/10
        else:
            livesp = int(lspace2)
        return livesp
    except:
        return()
    

# Площадь кухни

In [9]:
def getKitsp (flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        kitsp = re.split('Площадь кухни|Санузел', table)[1]
        kitsp1 = re.split('-|\n', kitsp)
        kitsp1 = re.split('\xa0|м2',kitsp1[2])
        kitsp2 = "".join([i for i in kitsp1[0] if i.isdigit()])
        if len(kitsp1[0]) != len(kitsp2):
            kitspace = int(kitsp2)/10
        else:
            kitspace = int(kitsp2)
        return kitspace
    except:
        return()
    

# Расстояние до центра

Центром условно будем считать Нулевой километр.
Расстояние просто рассчитаем по теореме Пифагора.

In [10]:
def getDist(flat_page):
    try:
        coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
        coords = re.split('&amp|center=|%2C', str(coords))
        coords_list = []
        for item in coords:
            if item[0].isdigit():
                coords_list.append(item)
        lat = float(coords_list[0])
        lon = float(coords_list[1])
        clat = 55.755831
        clon = 37.617673
        dist = ((clat-lat)**2 + (clon-lon)**2)**0.5
        dist = round(dist*111.134861111,5)
        return dist
    except:
        ()

# Расстояние до метро в минутах

In [11]:
def getMetrdist (flat_page):
    try:
        metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'}).contents[0]
        metrdist1 = re.split('\n',metrdist)
        return int(metrdist1[1].strip())
    except:
        return ()

# Пешая доступность метро

In [14]:
def getWalk (flat_page):
    try:
        metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'}).contents[0]
        walk = re.split('\n',metrdist)
        walk = walk[4].strip()
        if walk == 'пешком':
            return 1
        else:
            return 0
    except:
        return ()

# Основной материал здания

In [15]:
def getBrick(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Brick = re.split('Тип дома|Тип продажи', table)[1]
        Brick1 = re.split('\n', Brick)
        if Brick1[3].strip() != '':
            Brick1 = Brick1[3].strip()
        if Brick1 == ('кирпичный' or 'жб' or 'кирпично-монолитный'):
            answer = 1
        elif Brick1 == ('монолитный'):
            answer = 1
        else:
            answer = 0
        return answer
    except: 
        return ()

# Наличие балкона/лоджии

In [16]:
def getBal(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        bal = re.split('Балкон|Лифт', table)[1]
        bal1 = re.split('\n', bal)
        if bal1[1][0].isdigit():
            return 1
        else: 
            return 0
    except: 
        return ()

# Этаж, на котором расположена квартира

In [17]:
def getFloor (flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        floor_nfl = re.split('Этаж|Тип дома', table)[1]
        floor = re.split('-|\n', floor_nfl)[2]
        floor = re.split('-|\n|/|\xa0', floor_nfl)[2]
        return int(floor.strip())
    except:
        return ()

# Количество этажей в доме

In [18]:
def getNfloors (flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        floor_nfl = re.split('Этаж|Тип дома', table)[1]
        nfloor = re.split('-|\n', floor_nfl)[2]
        nfloor = re.split('-|\n|/|\xa0', floor_nfl)[5]
        return int(nfloor)
    except:
        nfloor = re.split('-|\n|/|\xa0', floor_nfl)[2]
        nfloor = "".join([i for i in nfloor if i.isdigit()])
        return int(nfloor)

# Новая ли квартира

In [19]:
def getNew (flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        New = re.split('Тип дома|Тип продажи', table)[1]
        New1 = re.split('\n', New)
        New1 = New1[2].strip()
        New1 = New1.replace(',','')
        if New1 == 'вторичка':
            return 0
        elif New1 == 'новостройка':
            return 1
        else:
            return ()
    except: 
        return ()

# Дальше нужно узнать, в каких районах собирать данные

In [20]:
#Через раздел расширенного поиска узнаем все основные районы (адимнистративные округа) Москвы и по порядку берем ссылки на квартиры из этого района
Names = ['ЦАО','САО', 'СВАО', 'ВАО','ЮВАО', 'ЮАО','ЮЗАО','ЗАО','СЗАО','ЗелАО','НАО', 'ТАО']
districts = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=152&district%5B1%5D=153&district%5B2%5D=154&district%5B3%5D=355&district%5B4%5D=356&district%5B5%5D=357&district%5B6%5D=358&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
           'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1']

In [21]:
def getPage(link):
    page = requests.get(link).content
    page = BeautifulSoup(page, 'lxml')
    return page

In [22]:

def getSubpageLink(link):
    return 'http://www.cian.ru/sale/flat/' + str(link) + '/'


In [23]:
from collections import OrderedDict #Для удаления повторений

In [24]:
def getLinks0(district, pages):
    for page in range(1, 30):
        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    return links

In [25]:
len(districts)

12

Значит всего 12 районов

## Сначала соберем все ссылки по районам

In [26]:
links = []
links_final = []
for i in range(len(districts)):
    links = []
    links = getLinks0(districts[i], pages = range(1,30,1))
    links_final.append(links)

### Теперь объединим все ссылки в один список

In [27]:
links_list = []
for i in range(len(links_final)):
    for j in range(len(links_final[i])):
        links_list.append(links_final[i][j])

In [40]:
len(links_list)

9048

#### Всего соберем ссылки на 9048 квартир

###### Проверяем:

In [28]:
link = links_final[0][1]
link

'149965274'

#### Функция, которая будет считать порядковый номер квартиры

In [29]:
def getN(number_iter):
    return number_iter + 1

### Итоговая функция, которая будет собирать данные с каждой квартиры и формировать таблицу

In [30]:
def ParseFlats(all_links):
    flatStats = pd.DataFrame(columns=['N', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp', 'Dist', 
                 'Metrdist', 'Walk', 'Brick', 'Bal', 'Floor', 'Nfloors', 'New'])
    count = 0
    imax = len(all_links)
    for i in range(len(all_links)):
        link = all_links[i]
        try: 
            flat_page = getPage(getSubpageLink(link))
            to_append = {'N': getN(count), 'Rooms': getRoom(flat_page), 'Price': getPrice(flat_page), 'Totsp': getTotsp(flat_page), 'Livesp': getLivesp(flat_page), 'Kitsp': getKitsp(flat_page), 'Dist': getDist(flat_page), 'Metrdist': getMetrdist(flat_page), 'Walk': getWalk(flat_page), 'Brick': getBrick(flat_page), 'Bal': getBal(flat_page), 'Floor': getFloor(flat_page), 'Nfloors': getNfloors(flat_page), 'New': getNew(flat_page)}
            flatStats = flatStats.append(to_append, ignore_index=True)
            count +=1 
            if count % 30 == 0:
                print('Я работаю над страницей номер {}'.format(int(count/30+1)))
        except:
            print('Whoops, something weird with link {}.'.format(link))
    print ('Я закончил')
    return flatStats

Деление на страницы здесь, по большому счету, условно, т. к. ссылки находятся в одном списке. Но воспользуемся им для более удобного контроля процесса

#### Приступаем к самому интригующему 

In [34]:
flat_stats = ParseFlats(links_list)

Я работаю над страницей номер 2
Я работаю над страницей номер 3
Я работаю над страницей номер 4
Я работаю над страницей номер 5
Я работаю над страницей номер 6
Я работаю над страницей номер 7
Я работаю над страницей номер 8
Я работаю над страницей номер 9
Я работаю над страницей номер 10
Я работаю над страницей номер 11
Я работаю над страницей номер 12
Я работаю над страницей номер 13
Я работаю над страницей номер 14
Я работаю над страницей номер 15
Я работаю над страницей номер 16
Я работаю над страницей номер 17
Я работаю над страницей номер 18
Я работаю над страницей номер 19
Я работаю над страницей номер 20
Я работаю над страницей номер 21
Я работаю над страницей номер 22
Я работаю над страницей номер 23
Я работаю над страницей номер 24
Я работаю над страницей номер 25
Я работаю над страницей номер 26
Я работаю над страницей номер 27
Я работаю над страницей номер 28
Я работаю над страницей номер 29
Я работаю над страницей номер 30
Я работаю над страницей номер 31
Я работаю над стра

#### Проверяем

In [39]:
flat_stats.head()

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Bal,Floor,Nfloors,New
0,1.0,2,69894300.0,80.2,56.6,5.7,3.35496,3,1,1.0,0.0,11.0,12.0,1.0
1,2.0,3,67800000.0,125.0,85.8,11.7,3.29872,6,1,0.0,1.0,5.0,9.0,0.0
2,3.0,3,129000000.0,138.4,(),(),5.60862,8,1,0.0,0.0,5.0,11.0,0.0
3,4.0,5,1730000.0,194.2,1,(),3.74535,9,1,0.0,1.0,10.0,13.0,0.0
4,5.0,1,4403057.0,22.7,22.7,(),6.42430,6,1,0.0,0.0,3.0,3.0,1.0


Эврика!

## Выгружаем в файл

In [36]:
flat_stats.to_csv('D:/CMF/ciandata1.csv')